In [3]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [40]:
cal_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
sales_df_wide = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [32]:
#selecting a random data point for prediction
input_data = sales_df_wide.sample()
print(input_data.head())

                                   id          item_id      dept_id  \
7009  HOUSEHOLD_1_354_CA_3_evaluation  HOUSEHOLD_1_354  HOUSEHOLD_1   

         cat_id store_id state_id  d_1  d_2  d_3  d_4  ...  d_1932  d_1933  \
7009  HOUSEHOLD     CA_3       CA    8    5    8    0  ...       2       3   

      d_1934  d_1935  d_1936  d_1937  d_1938  d_1939  d_1940  d_1941  
7009       2       2       0       5       5       0       9       1  

[1 rows x 1947 columns]


In [33]:
def final_fun_1(input_data):
    global cal_df,prices_df,sales_df_wide
    #there are many NaN values in event_name and event_type fields
    #Filling them with 'NoEvent' and 'None' resp
    cal_df['event_name_1'].fillna('NoEvent',inplace=True)
    cal_df['event_type_1'].fillna('None',inplace=True)
    cal_df['event_name_2'].fillna('NoEvent',inplace=True)
    cal_df['event_type_2'].fillna('None',inplace=True)
    #dropping redundant columns and changing datatypes to reduce memory usage
    cal_df.drop(['date','weekday'],axis=1,inplace=True)
    cal_df['wm_yr_wk'] = cal_df.wm_yr_wk.astype('int16')
    cal_df['d'] = cal_df.d.str[2:].astype('int16')
    for col in ['wday','month','snap_CA','snap_TX','snap_WI']:
        cal_df[col] = cal_df[col].astype('int8')
    for col in ['event_name_1','event_type_1','event_name_2','event_type_2']:
        cal_df[col] = cal_df[col].astype('category')
        
    # changing datatypes to reduce memory usage
    prices_df['store_id'] = prices_df.store_id.astype('category')
    prices_df['item_id'] = prices_df.item_id.astype('category')
    prices_df['wm_yr_wk'] = prices_df.wm_yr_wk.astype('int16')
    prices_df['sell_price'] = prices_df.sell_price.astype('float16')
    
    #converting the wide-form data frame to long-form so that columns from other 2 data frames can be merged
    train_df = pd.melt(sales_df_wide,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold')
    
    input_data = pd.melt(input_data,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold') 
    
    
    # changing datatypes to reduce memory usage
    train_df['d'] = train_df.d.str[2:].astype('int16')
    input_data['d'] = input_data.d.str[2:].astype('int16')
    
    train_df['units_sold'] = train_df.units_sold.astype('int16')
    input_data['units_sold'] = input_data.units_sold.astype('int16')
    
    for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
        train_df[col] = train_df[col].astype('category') 
        input_data[col] = input_data[col].astype('category') 
    
    
    #merging with 'cal_df'
    train_df = pd.merge(train_df,cal_df,on='d',how='left')
    input_data = pd.merge(input_data,cal_df,on='d',how='left')
    
    #merging with 'prices_df'
    train_df = pd.merge(train_df,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')
    input_data = pd.merge(input_data,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')
    
    del cal_df,prices_df,sales_df_wide

    #sell_price data is not available for many rows. 
    #For previous weeks filling this data by mean sell_prices for the item_id and store_id pair
    train_df['sell_price'].fillna(train_df.groupby(['store_id','item_id'])['sell_price'].transform('mean'),inplace=True)

    #calculating lag features and filling NA values with 0
    for i in [7,14,21,28]:
        train_df['lag_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.shift(i))
        train_df['lag_'+str(i)].fillna(0,inplace=True)

    #ref:https://www.kaggle.com/kyakovlev/m5-lags-features
    #rolling window mean features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_mean_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).mean())
        train_df['rolling_mean_'+str(i)].fillna(0,inplace=True)
        
    #rolling window meadion features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_median_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).median())
        train_df['rolling_median_'+str(i)].fillna(0,inplace=True)

    #dropping features 'wm_yr_wk' and 'year' as they are similar to 'd'
    train_df.drop(['wm_yr_wk','year'],axis=1,inplace=True)
    input_data.drop(['wm_yr_wk','year'],axis=1,inplace=True)
    
    for i in ['lag_7','lag_14','lag_21','lag_28','rolling_mean_7','rolling_mean_14','rolling_median_7','rolling_median_14']:
        train_df[i] = train_df[i].astype('float16')
      
    input_data = input_data.loc[input_data.d.isin(range(1914,1942))]
    input_data = pd.merge(input_data,train_df,on=['id','d'],how='left',suffixes=(False,'_x'))
    
    rename = {'item_idFalse':'item_id', 'dept_idFalse':'dept_id', 'cat_idFalse':'cat_id', \
             'store_idFalse':'store_id', 'state_idFalse':'state_id','units_soldFalse':'units_sold',\
             'wdayFalse':'wday', 'monthFalse':'month', 'event_name_1False':'event_name_1',\
             'event_type_1False':'event_type_1', 'event_name_2False':'event_name_2',\
             'event_type_2False':'event_type_2', 'snap_CAFalse':'snap_CA', 'snap_TXFalse':'snap_TX',\
             'snap_WIFalse':'snap_WI', 'sell_priceFalse': 'sell_price'}
    
    input_data.rename(columns=rename,inplace=True)
    
    input_data.drop(['item_id_x','dept_id_x','cat_id_x','store_id_x','state_id_x','units_sold_x',\
                     'wday_x','month_x','event_name_1_x','event_type_1_x','event_name_2_x',\
                     'event_type_2_x','snap_CA_x','snap_TX_x','snap_WI_x','sell_price_x','units_sold'],\
                    axis=1,inplace=True)
    
    train_df.drop(['units_sold'],axis=1,inplace=True)
    train_df = train_df.loc[(train_df.d.isin(range(1000,1914)))]

    #feature encoding
    le = LabelEncoder()
    le.fit(train_df.item_id.values)
    input_data['item_id'] = le.transform(input_data.item_id.values)
    
    le_id = LabelEncoder()
    le_id.fit(train_df.id.values)
    input_data['id'] = le_id.transform(input_data.id.values)
    
    le.fit(train_df.event_name_1.values)
    input_data['event_name_1'] = le.transform(input_data.event_name_1.values)
    
    le.fit(train_df.event_type_1.values)
    input_data['event_type_1'] = le.transform(input_data.event_type_1.values)
    
    le.fit(train_df.event_name_2.values)
    input_data['event_name_2'] = le.transform(input_data.event_name_2.values)
    
    le.fit(train_df.event_type_2.values)
    input_data['event_type_2'] = le.transform(input_data.event_type_2.values)

    le.fit(train_df.dept_id.values)
    input_data['dept_id'] = le.transform(input_data.dept_id.values)
    
    le.fit(train_df.cat_id.values)
    input_data['cat_id'] = le.transform(input_data.cat_id.values)
    
    le.fit(train_df.store_id.values)
    input_data['store_id'] = le.transform(input_data.store_id.values)
    
    le.fit(train_df.state_id.values)
    input_data['state_id'] = le.transform(input_data.state_id.values)
    
    for i in ['item_id','event_name_1','event_type_1','event_name_2','event_type_2']:
        input_data[i] = input_data[i].astype('int16')
    
    for i in ['dept_id','cat_id','store_id','state_id']:
        input_data[i] = input_data[i].astype('int8')
        
    del train_df
    
    with open('../input/pickledata/Pickle_CAT.pkl', 'rb') as file:  
        cat = pickle.load(file)
    y_pred_cat = cat.predict(input_data)
    
    with open('../input/pickledata/Pickle_DT.pkl', 'rb') as file:  
        dt = pickle.load(file)
    y_pred_dt = dt.predict(input_data)    

    with open('../input/pickledata/Pickle_LGBM.pkl', 'rb') as file:  
        lgbm = pickle.load(file)
    y_pred_lgbm = lgbm.predict(input_data)
    
    with open('../input/pickledata/Pickle_RF.pkl', 'rb') as file:  
        rf = pickle.load(file)
    y_pred_rf = rf.predict(input_data)    

    with open('../input/pickledata/Pickle_XGB.pkl', 'rb') as file:  
        xgb = pickle.load(file)
    y_pred_xgb = xgb.predict(input_data)
    
    x_test_meta = np.vstack((y_pred_dt,y_pred_rf,y_pred_xgb,y_pred_lgbm,y_pred_cat)).T
    del y_pred_dt,y_pred_rf,y_pred_xgb,y_pred_lgbm,y_pred_cat
    with open('../input/pickledata/Pickle_LR.pkl', 'rb') as file:  
        lr = pickle.load(file)
    y_pred_lr = lr.predict(x_test_meta)
    
    d = {'id':le_id.inverse_transform(input_data['id'].values),'d':input_data['d'].values,'units':y_pred_lr}
    y_pred = pd.DataFrame(data=d)
    y_pred = y_pred.pivot_table(index='id',columns='d',values='units').reset_index()
    names = {1942:'F1',1943:'F2',1944:'F3',1945:'F4',1946:'F5',1947:'F6',1948:'F7',1949:'F8',1950:'F9',1951:'F10',1952:'F11',1953:'F12',1954:'F13',1955:'F14',1956:'F15',1957:'F16',1958:'F17',1959:'F18',1960:'F19',1961:'F20',1962:'F21',1963:'F22',1964:'F23',1965:'F24',1966:'F25',1967:'F26',1968:'F27',1969:'F28'}
    y_pred.rename(columns=names,inplace=True)
    del d
    return y_pred

In [34]:
predictions = final_fun_1(input_data)
print(predictions)

d                               id      1914     1915      1916      1917  \
0  HOUSEHOLD_1_354_CA_3_evaluation  2.221088  1.91331  2.606695  2.529928   

d      1918      1919      1920      1921      1922  ...      1932      1933  \
0  2.299988  2.991133  3.493998  2.393302  2.573507  ...  1.647694  2.031491   

d      1934      1935      1936      1937      1938      1939      1940  \
0  2.269557  1.950168  1.458912  1.928056  2.459041  2.360754  4.042142   

d      1941  
0  3.961977  

[1 rows x 29 columns]


In [35]:
#function to calculate assymmetric rmse,custom metric function
def armse(y_act,y_pred):
    score=0
    n = len(y_act)
    diff = np.array(y_pred) - np.array(y_act)
    for ele in diff:
        if ele<0:
            score += 4*(ele**2)
        else:
            score += ele**2
    return np.sqrt(score/n)

In [37]:
def final_fun_2(input_data,y):
    global cal_df,prices_df,sales_df_wide
    #there are many NaN values in event_name and event_type fields
    #Filling them with 'NoEvent' and 'None' resp
    cal_df['event_name_1'].fillna('NoEvent',inplace=True)
    cal_df['event_type_1'].fillna('None',inplace=True)
    cal_df['event_name_2'].fillna('NoEvent',inplace=True)
    cal_df['event_type_2'].fillna('None',inplace=True)
    #dropping redundant columns and changing datatypes to reduce memory usage
    cal_df.drop(['date','weekday'],axis=1,inplace=True)
    cal_df['wm_yr_wk'] = cal_df.wm_yr_wk.astype('int16')
    cal_df['d'] = cal_df.d.str[2:].astype('int16')
    for col in ['wday','month','snap_CA','snap_TX','snap_WI']:
        cal_df[col] = cal_df[col].astype('int8')
    for col in ['event_name_1','event_type_1','event_name_2','event_type_2']:
        cal_df[col] = cal_df[col].astype('category')
        
    # changing datatypes to reduce memory usage
    prices_df['store_id'] = prices_df.store_id.astype('category')
    prices_df['item_id'] = prices_df.item_id.astype('category')
    prices_df['wm_yr_wk'] = prices_df.wm_yr_wk.astype('int16')
    prices_df['sell_price'] = prices_df.sell_price.astype('float16')
    
    #converting the wide-form data frame to long-form so that columns from other 2 data frames can be merged
    train_df = pd.melt(sales_df_wide,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold')
    
    input_data = pd.melt(input_data,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold') 
    
    
    # changing datatypes to reduce memory usage
    train_df['d'] = train_df.d.str[2:].astype('int16')
    input_data['d'] = input_data.d.str[2:].astype('int16')
    
    train_df['units_sold'] = train_df.units_sold.astype('int16')
    input_data['units_sold'] = input_data.units_sold.astype('int16')
    
    for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
        train_df[col] = train_df[col].astype('category') 
        input_data[col] = input_data[col].astype('category') 
    
    
    #merging with 'cal_df'
    train_df = pd.merge(train_df,cal_df,on='d',how='left')
    input_data = pd.merge(input_data,cal_df,on='d',how='left')
    
    #merging with 'prices_df'
    train_df = pd.merge(train_df,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')
    input_data = pd.merge(input_data,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')
    
    del cal_df,prices_df,sales_df_wide

    #sell_price data is not available for many rows. 
    #For previous weeks filling this data by mean sell_prices for the item_id and store_id pair
    train_df['sell_price'].fillna(train_df.groupby(['store_id','item_id'])['sell_price'].transform('mean'),inplace=True)

    #calculating lag features and filling NA values with 0
    for i in [7,14,21,28]:
        train_df['lag_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.shift(i))
        train_df['lag_'+str(i)].fillna(0,inplace=True)

    #ref:https://www.kaggle.com/kyakovlev/m5-lags-features
    #rolling window mean features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_mean_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).mean())
        train_df['rolling_mean_'+str(i)].fillna(0,inplace=True)
        
    #rolling window meadion features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_median_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).median())
        train_df['rolling_median_'+str(i)].fillna(0,inplace=True)

    #dropping features 'wm_yr_wk' and 'year' as they are similar to 'd'
    train_df.drop(['wm_yr_wk','year'],axis=1,inplace=True)
    input_data.drop(['wm_yr_wk','year'],axis=1,inplace=True)
    
    for i in ['lag_7','lag_14','lag_21','lag_28','rolling_mean_7','rolling_mean_14','rolling_median_7','rolling_median_14']:
        train_df[i] = train_df[i].astype('float16')
      
    input_data = input_data.loc[input_data.d.isin(range(1914,1942))]
    input_data = pd.merge(input_data,train_df,on=['id','d'],how='left',suffixes=(False,'_x'))
    
    rename = {'item_idFalse':'item_id', 'dept_idFalse':'dept_id', 'cat_idFalse':'cat_id', \
             'store_idFalse':'store_id', 'state_idFalse':'state_id','units_soldFalse':'units_sold',\
             'wdayFalse':'wday', 'monthFalse':'month', 'event_name_1False':'event_name_1',\
             'event_type_1False':'event_type_1', 'event_name_2False':'event_name_2',\
             'event_type_2False':'event_type_2', 'snap_CAFalse':'snap_CA', 'snap_TXFalse':'snap_TX',\
             'snap_WIFalse':'snap_WI', 'sell_priceFalse': 'sell_price'}
    
    input_data.rename(columns=rename,inplace=True)
    
    input_data.drop(['item_id_x','dept_id_x','cat_id_x','store_id_x','state_id_x','units_sold_x',\
                     'wday_x','month_x','event_name_1_x','event_type_1_x','event_name_2_x',\
                     'event_type_2_x','snap_CA_x','snap_TX_x','snap_WI_x','sell_price_x','units_sold'],\
                    axis=1,inplace=True)
    
    train_df.drop(['units_sold'],axis=1,inplace=True)
    train_df = train_df.loc[(train_df.d.isin(range(1000,1914)))]

    #feature encoding
    le = LabelEncoder()
    le.fit(train_df.item_id.values)
    input_data['item_id'] = le.transform(input_data.item_id.values)
    
    le_id = LabelEncoder()
    le_id.fit(train_df.id.values)
    input_data['id'] = le_id.transform(input_data.id.values)
    
    le.fit(train_df.event_name_1.values)
    input_data['event_name_1'] = le.transform(input_data.event_name_1.values)
    
    le.fit(train_df.event_type_1.values)
    input_data['event_type_1'] = le.transform(input_data.event_type_1.values)
    
    le.fit(train_df.event_name_2.values)
    input_data['event_name_2'] = le.transform(input_data.event_name_2.values)
    
    le.fit(train_df.event_type_2.values)
    input_data['event_type_2'] = le.transform(input_data.event_type_2.values)

    le.fit(train_df.dept_id.values)
    input_data['dept_id'] = le.transform(input_data.dept_id.values)
    
    le.fit(train_df.cat_id.values)
    input_data['cat_id'] = le.transform(input_data.cat_id.values)
    
    le.fit(train_df.store_id.values)
    input_data['store_id'] = le.transform(input_data.store_id.values)
    
    le.fit(train_df.state_id.values)
    input_data['state_id'] = le.transform(input_data.state_id.values)
    
    for i in ['item_id','event_name_1','event_type_1','event_name_2','event_type_2']:
        input_data[i] = input_data[i].astype('int16')
    
    for i in ['dept_id','cat_id','store_id','state_id']:
        input_data[i] = input_data[i].astype('int8')
        
    del train_df
    
    with open('../input/pickledata/Pickle_CAT.pkl', 'rb') as file:  
        cat = pickle.load(file)
    y_pred_cat = cat.predict(input_data)
    
    with open('../input/pickledata/Pickle_DT.pkl', 'rb') as file:  
        dt = pickle.load(file)
    y_pred_dt = dt.predict(input_data)    

    with open('../input/pickledata/Pickle_LGBM.pkl', 'rb') as file:  
        lgbm = pickle.load(file)
    y_pred_lgbm = lgbm.predict(input_data)
    
    with open('../input/pickledata/Pickle_RF.pkl', 'rb') as file:  
        rf = pickle.load(file)
    y_pred_rf = rf.predict(input_data)    

    with open('../input/pickledata/Pickle_XGB.pkl', 'rb') as file:  
        xgb = pickle.load(file)
    y_pred_xgb = xgb.predict(input_data)
    
    x_test_meta = np.vstack((y_pred_dt,y_pred_rf,y_pred_xgb,y_pred_lgbm,y_pred_cat)).T
    del y_pred_dt,y_pred_rf,y_pred_xgb,y_pred_lgbm,y_pred_cat
    with open('../input/pickledata/Pickle_LR.pkl', 'rb') as file:  
        lr = pickle.load(file)
        
    y_pred_lr = lr.predict(x_test_meta)
    
    armse_score = armse(y,y_pred_lr)
    rmse_score = mse(y,y_pred_lr,squared=False)
    return armse_score,rmse_score

In [38]:
lst = list(range(1914,1942))
lst = ['d_'+str(x) for x in lst]
y = []
for i,row in input_data.iterrows():
    y.extend(row[lst])

In [41]:
armse_score,rmse_score = final_fun_2(input_data,y)
print("ARMSE: {}, RMSE: {}".format(armse_score,rmse_score))

ARMSE: 3.432830810585946, RMSE: 2.000308224671881
